# **Zadanie 1**

Zaimplementuj w środowisku MPI obliczanie równoległe liczby PI z wzoru Leibniz-a.

In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>
 

// funkcja potęgująca bo funkcja w math.h pow nie działa 

double potegowanie(double a , int b )
{
      double result=1.0;
      for (int i = 1; i <= b; i++)
      {
          result *= a;
      }
      return result; 
}

int main(int argc, char **argv)
{
    int numer_procesu;
    int liczba_procesow;
    int tag = 1;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);
    clock_t t;
    double time_taken;

    if(numer_procesu == 0)
    {
      double lpi = 0 ,x = 0,total_time =0,t=0;

      for(int i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        lpi +=x;
        MPI_Recv(&t, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        total_time+=t;
      }
      printf("Wartosc liczby pi = %f \n", 4 * lpi);
      printf("Total time taken = %f \n", total_time);
    }

    if(numer_procesu != 0)
    {
      t = clock();
      double x = potegowanie( -1, numer_procesu - 1) / ( 2 * numer_procesu - 1); 
      printf("Wynik procesu %d = %f \t" , numer_procesu , x);
      MPI_Send(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      t = clock() - t;
      time_taken = ((double)t)/CLOCKS_PER_SEC;
      MPI_Send(&time_taken, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      printf("Czas wykonywania sie procesu nr %d = %f\n",numer_procesu,time_taken);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 60 --allow-run-as-root a.out

Wynik procesu 15 = 0.034483 	Czas wykonywania sie procesu nr 15 = 0.000075
Wynik procesu 54 = -0.009346 	Czas wykonywania sie procesu nr 54 = 0.000069
Wynik procesu 39 = 0.012987 	Czas wykonywania sie procesu nr 39 = 0.000036
Wynik procesu 40 = -0.012658 	Czas wykonywania sie procesu nr 40 = 0.000077
Wynik procesu 7 = 0.076923 	Czas wykonywania sie procesu nr 7 = 0.000037
Wynik procesu 41 = 0.012346 	Czas wykonywania sie procesu nr 41 = 0.000037
Wynik procesu 20 = -0.025641 	Czas wykonywania sie procesu nr 20 = 0.000048
Wynik procesu 19 = 0.027027 	Czas wykonywania sie procesu nr 19 = 0.000045
Wynik procesu 23 = 0.022222 	Czas wykonywania sie procesu nr 23 = 0.000029
Wynik procesu 44 = -0.011494 	Czas wykonywania sie procesu nr 44 = 0.000044
Wynik procesu 28 = -0.018182 	Czas wykonywania sie procesu nr 28 = 0.000052
Wynik procesu 6 = -0.090909 	Czas wykonywania sie procesu nr 6 = 0.000037
Wynik procesu 53 = 0.009524 	Czas wykonywania sie procesu nr 53 = 0.000038
Wynik procesu 3 = 0.200

# **Zadanie 2**

Zaimplementuj w środowisku MPI wyznaczenie numerycznej wartość całki y=f(x) (postać funkcji 
wybierasz sam) w przedziale <a,b> przy pomocy N trapezów.
Proces 0, to proces Root, który zarządza obliczeniami:
- wysyła do procesów roboczych Slave początek a_local i koniec b_local lokalnego przedziału 
całkowania dla danego procesu oraz liczbę N_local trapezów, z których należy policzyć całkę
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
// moja funkcja
double funkcja(double x)
{
  return x*x;
}

int main(int argc, char **argv)
{
    int numer_procesu;
    int liczba_procesow;
    int tag=1;
    
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);
    clock_t t;
    double time_taken;

    double l_procesow = liczba_procesow, n_procesu = numer_procesu;

    double granica_dolna_calki = 1,granica_gorna_calki = 4;
    double h = (granica_gorna_calki - granica_dolna_calki) / l_procesow;
    
    if(numer_procesu == 0)
    {
      double calka = 0 ,x=0;
      for(int i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        calka +=x;
      }
      printf("Wartosc funkcji(dolnej granicy calki)/2 * h = %f \n",funkcja(granica_dolna_calki)/2 * h);
      printf("Wartosc funkcji(gornej granicy calki)/2 * h = %f \n",funkcja(granica_gorna_calki)/2 * h);

      printf("Wartosc calki = %1f \n", h * (funkcja(granica_dolna_calki)/2 + calka 
        + funkcja(granica_gorna_calki)/2));
    }

    if(numer_procesu != 0)
    {
        t = clock();
        double xi,wynik;
        n_procesu = numer_procesu;
        xi = granica_dolna_calki + (n_procesu/l_procesow) 
            * (granica_gorna_calki - granica_dolna_calki);
        wynik = funkcja(xi);
        printf("Wynik procesu %d = %f \t", numer_procesu,wynik*h);
        MPI_Send(&wynik, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
        t = clock() - t;
        time_taken = ((double)t)/CLOCKS_PER_SEC;
        printf("Czas wykonywania sie procesu nr %d = %f\n",numer_procesu,time_taken);
    }

    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 100 --allow-run-as-root a.out

Wynik procesu 46 = 0.169932 	Czas wykonywania sie procesu nr 46 = 0.000096
Wynik procesu 38 = 0.137388 	Czas wykonywania sie procesu nr 38 = 0.000076
Wynik procesu 42 = 0.153228 	Czas wykonywania sie procesu nr 42 = 0.000054
Wynik procesu 79 = 0.340707 	Czas wykonywania sie procesu nr 79 = 0.000056
Wynik procesu 96 = 0.451632 	Czas wykonywania sie procesu nr 96 = 0.000045
Wynik procesu 57 = 0.220323 	Czas wykonywania sie procesu nr 57 = 0.000054
Wynik procesu 3 = 0.035643 	Czas wykonywania sie procesu nr 3 = 0.000050
Wynik procesu 39 = 0.141267 	Czas wykonywania sie procesu nr 39 = 0.000043
Wynik procesu 36 = 0.129792 	Czas wykonywania sie procesu nr 36 = 0.000048
Wynik procesu 9 = 0.048387 	Czas wykonywania sie procesu nr 9 = 0.000050
Wynik procesu 26 = 0.095052 	Czas wykonywania sie procesu nr 26 = 0.000050
Wynik procesu 90 = 0.410700 	Czas wykonywania sie procesu nr 90 = 0.000043
Wynik procesu 98 = 0.465708 	Czas wykonywania sie procesu nr 98 = 0.000054
Wynik procesu 15 = 0.063075 	